In [2]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Loading the dam-specific dataset
df = pd.read_excel("../data/processed/dam_occupancy_cleaned.csv")

# Converting the 'Tarih' column to datetime using automatic detection with dayfirst
df['Tarih'] = pd.to_datetime(df['Tarih'], dayfirst=True)
df = df.set_index('Tarih')

# Filling any missing values by forward fill method (ensuring clean dataset)
df = df.fillna(method='ffill')

# Defining look-back period
look_back = 30

# Scaling the data using MinMaxScaler (fitting across all columns)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# Creating multivariate sequences
def create_multivariate_sequences(data, look_back=30):
    X, y = [], []
    for i in range(look_back, len(data)):
        X.append(data[i-look_back:i, :])
        y.append(data[i, :])  # multi-output: predicting all dams at once
    return np.array(X), np.array(y)

# Creating sequences
X, y = create_multivariate_sequences(scaled_data, look_back)

# Splitting train and test (last 30 days as test)
X_train, y_train = X[:-30], y[:-30]
X_test, y_test = X[-30:], y[-30:]

# Building the LSTM model (multi-input, multi-output)
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(y_train.shape[1]))  # output dimension = number of dams
model.compile(optimizer='adam', loss='mse')

# Training the model
model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1)

# Making predictions on the test set
predictions = model.predict(X_test)
predictions_inverse = scaler.inverse_transform(predictions)
y_test_inverse = scaler.inverse_transform(y_test)

# Calculating metrics (average across all dams)
mae_lstm_multi = mean_absolute_error(y_test_inverse, predictions_inverse)
rmse_lstm_multi = np.sqrt(mean_squared_error(y_test_inverse, predictions_inverse))

print(f"Multivariate LSTM MAE (Test Set): {mae_lstm_multi:.2f}")
print(f"Multivariate LSTM RMSE (Test Set): {rmse_lstm_multi:.2f}")

# Plotting one example dam (e.g., Ömerli)
dam_index = df.columns.get_loc('Ömerli')
plt.figure(figsize=(14,5))
plt.plot(df.index[-30:], y_test_inverse[:, dam_index], label='Actual Ömerli', color='orange')
plt.plot(df.index[-30:], predictions_inverse[:, dam_index], label='Multivariate LSTM Forecast', color='purple')
plt.title("Multivariate LSTM Forecast vs Actual - Ömerli")
plt.xlabel("Date")
plt.ylabel("Occupancy Rate (%)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


ValueError: Excel file format cannot be determined, you must specify an engine manually.